<div><a href="https://knodis-research-group.github.io/"><img style="float: right; width: 128px; vertical-align:middle" src="https://knodis-research-group.github.io/knodis-logo_horizontal.png" alt="KNODIS logo" /></a>

# Regularización _dropout_<a id="top"></a>

<i><small>Última actualización: 2025-03-14</small></i></div>

***

## Introducción

Vamos a reflexionar un poco sobre lo que se espera de un buen modelo predictivo: queremos que tenga un buen rendimiento con datos nunca vistos durante el entrenamientose desempeñe bien con datos no vistos. La teoría clásica de la generalización siempre ha sugerido que, para tener modelos más generalistas, debemos apuntar hacia modelos lo más simples posible.

Sin embargo, en 2014, [Srivastava et al.](https://www.jmlr.org/papers/volume15/srivastava14a/srivastava14a.pdf) propusieron un nuevo punto de vista para generalizar. Curiosamente, la analogía fue hecha por ellos con la reproducción sexual. Los autores argumentaron que el sobreajuste de las redes neuronales se caracteriza por un estado en el cual cada capa depende de un patrón específico de activaciones de la capa anterior, a lo cual llaman coadaptación. El _dropout_, según ellos, rompe esta coadaptación de la misma manera que la reproducción sexual rompe los genes coadaptados.

El **_dropout_** es una técnica de regularización utilizada para prevenir el sobreajuste en las redes neuronales. Durante el entrenamiento, implica «apagar» algunos neuronas seleccionadas al azar en la red en cada iteración. Esto fuerza a la red a **que todas las neuronas aprendan de todos los ejemplos**, haciéndola más robusta y menos propensa al sobreajuste.

<center>
<figure class="image">
    <img src="https://blazaid.github.io/aprendizaje-profundo/Slides/images/dropout.png" alt="Esquema de un MLP antes y después del Dropout" />
    <figcaption><em><strong>Figura 1.</strong>Esquema de un MLP antes y después del Dropout.</em></figcaption>
</figure>
</center>

Lo normal es que una vez terminado el entrenamiento el _dropout_ se desactive, ya que la idea es que reparta el conocimiento a través de todas las neuronas durante el entrenamiento. No obstante, hay algunas excepciones: algunos autores utilizan el _dropout_ también durante el cálculo de la precisión con el conjunto de prueba como una heurística para la incertidumbre de las predicciones de la red neuronal. Es decir, si las predicciones coinciden en muchas diferentes permutaciones de _dropout_, entonces podríamos afirmar con cierta confianza que nuestro modelo es más robusto.

## Objetivos

El propósito será implementar un modelo de perceptrón multicapa bastante grande, que sea capaz de clasificar los datos en el conjunto de datos mnist, pero que haya sido regularizado con _dropout_ para intentar asegurar que su potencia no afecte su capacidad de generalización.

## Bibliotecas y configuración

A continuación, importaremos las bibliotecas que se utilizarán a lo largo del cuaderno.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torchvision

También configuraremos algunos parámetros para adaptar la presentación gráfica.

In [ ]:
%matplotlib inline
plt.style.use('ggplot')
plt.rcParams.update({'figure.figsize': (20, 6),'figure.dpi': 64})

Por último, establecemos las constantes de los recursos comunes.

In [ ]:
DATASETS_DIR = './tmp'

***

## Preparación de datos

Usaremos el conjunto de datos `mnist` que hemos usado previamente y que nos viene de maravilla para este propósito.

In [ ]:
train_set = torchvision.datasets.MNIST(
    root=DATASETS_DIR,
    train=True,
    download=True,
    transform=torchvision.transforms.ToTensor(),
)
validation_set = torchvision.datasets.MNIST(
    root=DATASETS_DIR,
    train=False,
    download=True,
    transform = torchvision.transforms.ToTensor(),
)

train_loader = torch.utils.data.DataLoader(
    dataset=train_set,
    batch_size=256,
    shuffle=True,
)
validation_loader = torch.utils.data.DataLoader(
    validation_set,
    batch_size=256,
    shuffle=False,
)

## Usando dropout en PyTorch

La verdad es que es bastante fácil. Todo lo que tenemos que hacer es añadir una capa `Dropout` después de la capa conectada a la cual queremos aplicar la regularización. El único parámetro que hay que pasar al constructor es la probabilidad de que una neurona sea o no desactivada.

Durante el entrenamiento, esta capa eliminará aleatoriamente las salidas de la capa anterior (y por lo tanto las entradas de la siguiente capa) de acuerdo con la probabilidad especificada. Cuando no está en modo de entrenamiento,  simplemente pasa los datos sin modificar.

Veamos primero cómo funcionarían dos modelos similares para el problema de `mnist`. Primero uno sin capa `Dropout`.

In [ ]:
model_no_dropout = torch.nn.Sequential(
    torch.nn.Flatten(),
    torch.nn.LazyLinear(out_features=1024),
    torch.nn.ReLU(),
    torch.nn.LazyLinear(out_features=1024),
    torch.nn.ReLU(),
    torch.nn.LazyLinear(out_features=10),
    torch.nn.ReLU(),
)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model_no_dropout.parameters())

Ahora entrenemos el modelo. Esto igual lleva un rato; después de todo es un modelo bastante grande.

In [ ]:
history = {
    'train_loss': [],
    'train_accuracy': [],
    'val_loss': [],
    'val_accuracy': [],
}

num_epochs = 100
for epoch in range(1, num_epochs + 1):
    # Entrenamiento
    model_no_dropout.train()  # Ponemos el modelo en modo entrenamiento
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model_no_dropout(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()  # accumulate loss
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    train_loss = running_loss / len(train_loader)
    train_accuracy = correct / total

    # Validación
    model_no_dropout.eval()  # Ponemos el modelo en modo evaluación
    val_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for inputs, labels in validation_loader:
            outputs = model_no_dropout(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()
    
    val_loss /= len(validation_loader)
    val_accuracy = correct_val / total_val

    history['train_loss'].append(train_loss)
    history['train_accuracy'].append(train_accuracy)
    history['val_loss'].append(val_loss)
    history['val_accuracy'].append(val_accuracy)

    print(f"Epoch {epoch} - "
          f"Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.4f} | "
          f"Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.4f}")

Veamos la evolución del entrenamiento

In [ ]:
pd.DataFrame(history).plot()
plt.xlabel('Epoch num.')
plt.yscale('log')
plt.show()

Y ahora uno con _dropout_. En Keras, implementar el dropout es sencillo. Se utiliza una capa especial llamada `Dropout`, que se inserta entre las capas consecutivas de la red neuronal. La capa Dropout toma un argumento llamado `rate`, que especifica la fracción de neuronas que se desactivarán en cada iteración. Aquí hay un ejemplo:

In [ ]:
model_dropout = torch.nn.Sequential(
    torch.nn.Flatten(),
    torch.nn.LazyLinear(out_features=1024),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.75),
    torch.nn.LazyLinear(out_features=1024),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.75),
    torch.nn.LazyLinear(out_features=10),
    torch.nn.ReLU(),
)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model_dropout.parameters())

En este ejemplo, hemos añadido dos capas de `Dropout` con una tasa del $50\%$. Esto significa que la mitad de las neuronas se desactivarán aleatoriamente durante cada _batch_ (no _apoch). Esto es un valor bastante común, pero no es el único. En general, el valor de la tasa de _dropout_ es un hiperparámetro que se puede ajustar para obtener el mejor rendimiento del modelo.

Entrenemos el modelo.

In [ ]:
history = {
    'train_loss': [],
    'train_accuracy': [],
    'val_loss': [],
    'val_accuracy': [],
}

num_epochs = 100
for epoch in range(1, num_epochs + 1):
    # Entrenamiento
    model_dropout.train()  # Ponemos el modelo en modo entrenamiento
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model_dropout(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()  # accumulate loss
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    train_loss = running_loss / len(train_loader)
    train_accuracy = correct / total

    # Validación
    model_dropout.eval()  # Ponemos el modelo en modo evaluación
    val_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for inputs, labels in validation_loader:
            outputs = model_dropout(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()
    
    val_loss /= len(validation_loader)
    val_accuracy = correct_val / total_val

    history['train_loss'].append(train_loss)
    history['train_accuracy'].append(train_accuracy)
    history['val_loss'].append(val_loss)
    history['val_accuracy'].append(val_accuracy)

    print(f"Epoch {epoch} - "
          f"Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.4f} | "
          f"Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.4f}")


Y veamos también su evolución:

In [ ]:
pd.DataFrame(history).plot()
plt.xlabel('Epoch num.')
plt.yscale('log')
plt.show()

Comparemos los valores obtenidos de los conjuntos de ambos modelos:

In [ ]:
def evaluate_model(model, data_loader, batch_size=64):
    model.eval()
    
    total_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for inputs, targets in data_loader:
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            total_loss += loss.item() * inputs.size(0)
            
            preds = outputs.argmax(dim=1)
            correct += (preds == targets).sum().item()
            total += inputs.size(0)
    
    avg_loss = total_loss / total
    accuracy = correct / total
    return avg_loss, accuracy

print('Without dropout:')
train_loss_1, train_accuracy_1 = evaluate_model(model_no_dropout, train_loader)
test_loss_1, test_accuracy_1 = evaluate_model(model_no_dropout, validation_loader)
print(f'\tLoss     -> Train: {train_loss_1}, Test: {test_loss_1}')
print(f'\tAccuracy -> Train: {train_accuracy_1}, Test: {test_accuracy_1}')

print('With dropout:')
train_loss_2, train_accuracy_2 = evaluate_model(model_dropout, train_loader)
test_loss_2, test_accuracy_2 = evaluate_model(model_dropout, validation_loader)
print(f'\tLoss     -> Train: {train_loss_2}, Test: {test_loss_2}')
print(f'\tAccuracy -> Train: {train_accuracy_2}, Test: {test_accuracy_2}')

El _dropout_ funciona forzando a la red neuronal a aprender características redundantes en distintos subconjuntos de neuronas. Durante el entrenamiento, la red neuronal se ve obligada a aprender características redundantes en diferentes subconjuntos de neuronas, lo que la hace más robusta y menos propensa al sobreajuste. La tendencia general observada es que en los modelos muy potentes el conocimiento tiende a repartirse entre todas las conexiones del modelo, lo que hace que se mitigue la sobreespecialización y aumente la generalización.

## Conclusiones

Hemos realizado una comparación entre dos modelos, uno sin _dropout_ y otro con él, y hemos demostrado que el uso del _dropout_ puede mejorar significativamente la capacidad predictiva de los modelos. Además, hemos explicado cómo funciona el _dropout_ y cómo se puede ajustar su hiperparámetro para optimizar su rendimiento.

El _dropout_ es una técnica muy eficaz para evitar el sobreajuste en las redes neuronales. Consiste en apagar aleatoriamente algunas neuronas de la red durante cada iteración de entrenamiento, forzando a la red a aprender características redundantes a través de diferentes subconjuntos de neuronas. En Keras, la implementación de dropout es sencilla utilizando la capa `Dropout`.

***

<div><img style="float: right; width: 120px; vertical-align:top" src="https://mirrors.creativecommons.org/presskit/buttons/88x31/png/by-nc-sa.png" alt="Creative Commons by-nc-sa logo" />

[Volver al inicio](#top)

</div>